In [1]:
from array import array
import reprlib
import math


class Vector:
    typecode = 'd'

    def __init__(self, components):
        self._components = array(self.typecode, components)  # 受保护的/私有属性

    def __iter__(self):
        return iter(self._components)  # 迭代器

    def __repr__(self):
        components = reprlib.repr(self._components)  # 有限长度表示形式
        components = components[components.find('['):-1]  # 去掉 array('d' 和 )
        return 'Vector({})'.format(components)

    def __str__(self):
        return str(tuple(self))

    def __bytes__(self):
        return (bytes([ord(self.typecode)]) +
                bytes(self._components))  # 直接从 _components 构建 bytes 对象

    def __eq__(self, other):
        return tuple(self) == tuple(other)

    def __abs__(self):
        return math.sqrt(sum(x * x for x in self))  # 先计算平方值和，再使用 sqrt 方法开平方

    def __bool__(self):
        return bool(abs(self))

    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(memv)  # 直接传入 memoryview

如果 Vector 实例的分量超过 6 个， repr() 生成的字符串就会用 ... 省略一部分

使用 reprlib 模块可以生成长度有限的表示形式，reprlib.repr 函数用于生成大型结构或递归结构的安全表示形式，它会限制输出字符串的长度，用 `...` 表示截断的部分。

调用 repr() 函数的目的是调试，因此绝对不能抛出异常。

In [2]:
Vector([3.1, 4.2])

In [3]:
Vector((3, 4, 5))

In [4]:
Vector(range(10))

在面向对象编程中，协议是非正式的接口，只在文档中定义，在代码中不定义。例如，Python 的序列协议只需要 \_\_len__ 和 \_\_getitem__ 两个方法。我们说它是序列，因为它的行为像序列。
- 这就是鸭子类型（duck typing）

协议是鸭子类型语言使用的非正式接口

In [5]:
import numbers


class Vector:
    typecode = 'd'

    def __init__(self, components):
        self._components = array(self.typecode, components)

    def __iter__(self):
        return iter(self._components)

    def __repr__(self):
        components = reprlib.repr(self._components)
        components = components[components.find('['):-1]
        return 'Vector({})'.format(components)

    def __str__(self):
        return str(tuple(self))

    def __bytes__(self):
        return (bytes([ord(self.typecode)]) +
                bytes(self._components))

    def __eq__(self, other):
        return tuple(self) == tuple(other)

    def __abs__(self):
        return math.sqrt(sum(x * x for x in self))

    def __bool__(self):
        return bool(abs(self))
    
    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(memv)
    
    # 序列协议
    def __len__(self):
        return len(self._components)
    
    # 序列协议
    def __getitem__(self, index):
        cls = type(self)  # 获取实例所属的类
        if isinstance(index, slice):  # 如果 index 是 slice 对象
            return cls(self._components[index])  # 调用类的构造方法
        elif isinstance(index, numbers.Integral):  # 如果是 int 或 其他整数类型
            return self._components[index]  # 返回 _components 中相应的元素
        else:
            msg = '{cls.__name__} indices must be integers'
            raise TypeError(msg.format(cls=cls))  # 否则抛出异常

大量使用 isinstance 可能表明面向对象设计得不好，不过在 \_\_getitem__ 方法中使用它处理切片是合理的。

在 isinstance 中使用抽象基类测试能让 API 更灵活且更容易更新。

In [6]:
v1 = Vector([3, 4, 5])
len(v1)

In [7]:
v7 = Vector(range(7))
v7[-1]

In [8]:
# 切片创建一个新的实例
v7[1:4]

In [9]:
# 长度为 1 的切片也创建一个 Vector 实例
v1[-1:]

In [10]:
# 不支持多维索引
v7[1, 2]

TypeError: Vector indices must be integers

In [11]:
class MySeq:
    def __getitem__(self, index):
        return index

In [12]:
s = MySeq()
s[1]

1

In [13]:
s[1:4]

slice(1, 4, None)

In [14]:
s[1:4:2]

slice(1, 4, 2)

In [15]:
# [] 中有逗号，__getitem__ 收到的是元组
s[1:4:2, 9]

(slice(1, 4, 2), 9)

In [16]:
# 元组中可以有多个切片对象
s[1:4:2, 7:9]

(slice(1, 4, 2), slice(7, 9, None))

slice 有个 indices 属性
- indices 方法开放了内置序列实现的棘手逻辑，用于优雅地处理缺失索引和负数索引，以及长度超过目标序列的切片、

In [17]:
# slice 是内置类型
dir(slice)

['__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'indices',
 'start',
 'step',
 'stop']

In [18]:
# 'ABCDE'[:10:2] = 'ABCDE'[0:5:2]
slice(None, 10, 2).indices(5)

(0, 5, 2)

In [19]:
# 'ABCDE'[-3:] = 'ABCDE'[2:5:1]
slice(-3, None, None).indices(5)

(2, 5, 1)

In [20]:
class Vector:
    typecode = 'd'

    def __init__(self, components):
        self._components = array(self.typecode, components)

    def __iter__(self):
        return iter(self._components)

    def __repr__(self):
        components = reprlib.repr(self._components)
        components = components[components.find('['):-1]
        return 'Vector({})'.format(components)

    def __str__(self):
        return str(tuple(self))

    def __bytes__(self):
        return (bytes([ord(self.typecode)]) +
                bytes(self._components))

    def __eq__(self, other):
        return tuple(self) == tuple(other)

    def __abs__(self):
        return math.sqrt(sum(x * x for x in self))

    def __bool__(self):
        return bool(abs(self))

    def __len__(self):
        return len(self._components)

    def __getitem__(self, index):
        cls = type(self)
        if isinstance(index, slice):
            return cls(self._components[index])
        elif isinstance(index, numbers.Integral):
            return self._components[index]
        else:
            msg = '{.__name__} indices must be integers'
            raise TypeError(msg.format(cls))

    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(memv)
    

    shortcut_names = 'xyzt'

    def __getattr__(self, name):
        cls = type(self)  # 获取 Vector
        if len(name) == 1:  # 属性名只有一个字母，可能是 shortcut_names 中的一个
            pos = cls.shortcut_names.find(name)  # 查找字母位置
            if 0 <= pos < len(self._components):  # 在范围内，返回对应元素
                return self._components[pos]
        msg = '{.__name__!r} object has no attribute {!r}'  # 指明标准的消息文本
        raise AttributeError(msg.format(cls, name))

In [21]:
v = Vector(range(5))
v

Vector([0.0, 1.0, 2.0, 3.0, 4.0])

In [22]:
v.x

0.0

In [23]:
# 赋值，应该抛出异常
v.x = 10
v.x

10

In [24]:
# 向量的分量未改变
v

Vector([0.0, 1.0, 2.0, 3.0, 4.0])

\_\_getattr__ 的运作方式：仅当对象没有指定名称的属性时，Python 才会调用那个方法，这是一种后备机制。

In [25]:
class Vector:
    typecode = 'd'

    def __init__(self, components):
        self._components = array(self.typecode, components)

    def __iter__(self):
        return iter(self._components)

    def __repr__(self):
        components = reprlib.repr(self._components)
        components = components[components.find('['):-1]
        return 'Vector({})'.format(components)

    def __str__(self):
        return str(tuple(self))

    def __bytes__(self):
        return (bytes([ord(self.typecode)]) +
                bytes(self._components))

    def __eq__(self, other):
        return tuple(self) == tuple(other)

    def __abs__(self):
        return math.sqrt(sum(x * x for x in self))

    def __bool__(self):
        return bool(abs(self))

    def __len__(self):
        return len(self._components)

    def __getitem__(self, index):
        cls = type(self)
        if isinstance(index, slice):
            return cls(self._components[index])
        elif isinstance(index, numbers.Integral):
            return self._components[index]
        else:
            msg = '{.__name__} indices must be integers'
            raise TypeError(msg.format(cls))
    
    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(memv)
    shortcut_names = 'xyzt'

    def __getattr__(self, name):
        cls = type(self)
        if len(name) == 1:
            pos = cls.shortcut_names.find(name)
            if 0 <= pos < len(self._components):
                return self._components[pos]
        msg = '{.__name__!r} object has no attribute {!r}'
        raise AttributeError(msg.format(cls, name))

    def __setattr__(self, name, value):
        cls = type(self)
        if len(name) == 1:  # 特别处理名称是单个字符的属性
            if name in cls.shortcut_names:  # 如果是 xyzt 中的一个，设置特殊的错误信息
                error = 'readonly attribute {attr_name!r}'
            elif name.islower():  # 如果是小写字母，为所有小写字母设置特殊的错误信息
                error = "can't set attributes 'a' to 'z' in {cls_name!r}"
            else:
                error = ''  # 否则，错误信息为空
            if error:  # 如果有错误信息，则抛出异常
                msg = error.format(cls_name=cls.__name__, attr_name=name)
                raise AttributeError(msg)
        super().__setattr__(name, value)  # 默认情况：在超类上调用 __setattr__ 方法，提供标准行为

In [26]:
v = Vector(range(5))
v

Vector([0.0, 1.0, 2.0, 3.0, 4.0])

In [27]:
v.x

0.0

In [28]:
v.x = 10.0
v

AttributeError: readonly attribute 'x'

super() 函数用于动态访问超类的方法。

不建议只为了避免创建实例而使用 \_\_slots__ 属性。\_\_slots__ 属性只应该用于节省内存，而且仅当内存严重不足时才应该这么做。

多数时候，如果实现了 \_\_getattr__ 方法，那么也要定义 \_\_setattr__ 方法，以防对象的行为不一致。

如果想允许修改分类
- 可以使用 \_\_setitem__ 方法，支持 v[0] = 1.1
- 实现 \_\_setattr__ 方法。支持 v.x = 1.1

In [29]:
# 5!
2 * 3 * 4 * 5

120

In [30]:
import functools

# 5!
functools.reduce(lambda a,b: a*b, range(1, 6))

120

In [31]:
# 0~5 累计异或
n = 0
for i in range(1, 6):
    n ^= i
n

1

In [32]:
# 0~5 累计异或
functools.reduce(lambda a,b: a^b, range(6))

1

In [33]:
import operator

# 0~5 累计异或
functools.reduce(operator.xor, range(6))

1

functools.reduce() 的关键思想是：把一系列值规约成单个值。reduce() 函数的第一个参数是接受两个参数的函数，第二个参数是一个可迭代对象。

使用 reduce() 函数时最好提供第三个参数（reduce(function, iterable, initializer)），如果序列为空，initializer 是返回的结果；否则，在归约中使用它作为第一个参数，因此应该使用恒等值
- +、|、^：0
- *、&：1

operator 模块以函数的形式提供了 Python 的全部中缀运算符，从而减少使用 lambda 表达式。

映射归约：把函数应用到各个元素上，生成一个新序列（映射，map），然后计算机聚合值（归约，reduce）

在 Python 3 中，map 函数是惰性的，它会创建一个生成器，按需产出结果因此能节省内存。

In [34]:
class Vector:
    typecode = 'd'

    def __init__(self, components):
        self._components = array(self.typecode, components)

    def __iter__(self):
        return iter(self._components)

    def __repr__(self):
        components = reprlib.repr(self._components)
        components = components[components.find('['):-1]
        return 'Vector({})'.format(components)

    def __str__(self):
        return str(tuple(self))

    def __bytes__(self):
        return (bytes([ord(self.typecode)]) +
                bytes(self._components))
    
    # 结合 __hash__ 一起使用
    def __eq__(self, other):
        return (len(self) == len(other) and  # 长度不一样，则不相等
                all(a == b for a, b in zip(self, other)))  # 计算聚合值
    
    # 结合 __eq__ 一起使用
    def __hash__(self):
        hashes = (hash(x) for x in self)  # 创建生成器表达式，惰性计算各个分量的散列值
        return functools.reduce(operator.xor, hashes, 0)  # 计算聚合的散列值

    def __abs__(self):
        return math.sqrt(sum(x * x for x in self))

    def __bool__(self):
        return bool(abs(self))

    def __len__(self):
        return len(self._components)

    def __getitem__(self, index):
        cls = type(self)
        if isinstance(index, slice):
            return cls(self._components[index])
        elif isinstance(index, numbers.Integral):
            return self._components[index]
        else:
            msg = '{cls.__name__} indices must be integers'
            raise TypeError(msg.format(cls=cls))

    shortcut_names = 'xyzt'

    def __getattr__(self, name):
        cls = type(self)
        if len(name) == 1:
            pos = cls.shortcut_names.find(name)
            if 0 <= pos < len(self._components):
                return self._components[pos]
        msg = '{.__name__!r} object has no attribute {!r}'
        raise AttributeError(msg.format(cls, name))

    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(memv)

In [35]:
v1 = Vector([3, 4])
v2 = Vector([3.1, 4.2])
v3 = Vector(range(6))

hash(v1), hash(v2), hash(v3)

(7, 384307168202284039, 1)

zip 函数生成一个由元组构成的生成器，元组中的元素来自参数传入的各个可迭代对象。一旦有一个输入耗尽，zip 函数会立即停止生成值，而且不发出警告。

使用 zip 函数能够轻松地并行迭代两个或更多可迭代对象，它返回的元组可以拆包成变量，分别对应各个并行输入中的一个元素。

In [36]:
# 返回生成器，按需生成元组
zip(range(3), 'ABC')

In [37]:
list(zip(range(3), 'ABC'))

[(0, 'A'), (1, 'B'), (2, 'C')]

In [38]:
# 当一个可迭代对象耗尽后，不发出警告就停止
list(zip(range(3), 'ABC', [0.0, 1.1, 2.2, 3.3]))

[(0, 'A', 0.0), (1, 'B', 1.1), (2, 'C', 2.2)]

In [39]:
from itertools import zip_longest

# 填充缺失值，直到最常的可迭代对象耗尽
list(zip_longest(range(3), 'ABC', [0.0, 1.1, 2.2, 3.3], fillvalue=-1))

[(0, 'A', 0.0), (1, 'B', 1.1), (2, 'C', 2.2), (-1, -1, 3.3)]

In [40]:
import itertools


class Vector:
    typecode = 'd'

    def __init__(self, components):
        self._components = array(self.typecode, components)

    def __iter__(self):
        return iter(self._components)

    def __repr__(self):
        components = reprlib.repr(self._components)
        components = components[components.find('['):-1]
        return 'Vector({})'.format(components)

    def __str__(self):
        return str(tuple(self))

    def __bytes__(self):
        return (bytes([ord(self.typecode)]) +
                bytes(self._components))

    def __eq__(self, other):
        return (len(self) == len(other) and
                all(a == b for a, b in zip(self, other)))

    def __hash__(self):
        hashes = (hash(x) for x in self)
        return functools.reduce(operator.xor, hashes, 0)

    def __abs__(self):
        return math.sqrt(sum(x * x for x in self))

    def __bool__(self):
        return bool(abs(self))

    def __len__(self):
        return len(self._components)

    def __getitem__(self, index):
        cls = type(self)
        if isinstance(index, slice):
            return cls(self._components[index])
        elif isinstance(index, numbers.Integral):
            return self._components[index]
        else:
            msg = '{.__name__} indices must be integers'
            raise TypeError(msg.format(cls))

    shortcut_names = 'xyzt'

    def __getattr__(self, name):
        cls = type(self)
        if len(name) == 1:
            pos = cls.shortcut_names.find(name)
            if 0 <= pos < len(self._components):
                return self._components[pos]
        msg = '{.__name__!r} object has no attribute {!r}'
        raise AttributeError(msg.format(cls, name))

    def angle(self, n):  # n 维球体，计算角坐标
        r = math.sqrt(sum(x * x for x in self[n:]))
        a = math.atan2(r, self[n-1])
        if (n == len(self) - 1) and (self[-1] < 0):
            return math.pi * 2 - a
        else:
            return a

    def angles(self):  # 生成器表达式，按需计算所有角坐标
        return (self.angle(n) for n in range(1, len(self)))
    
    # 格式化
    def __format__(self, fmt_spec=''):
        if fmt_spec.endswith('h'):  # 超球面坐标
            fmt_spec = fmt_spec[:-1]
            coords = itertools.chain([abs(self)],
                                     self.angles())  # 生成生成器表达式
            outer_fmt = '<{}>'  # 球面坐标
        else:
            coords = self
            outer_fmt = '({})'  # 笛卡尔坐标
        components = (format(c, fmt_spec) for c in coords)  # 创建生成器表达式，按需格式化各个坐标元素
        return outer_fmt.format(', '.join(components))  # 格式化

    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(memv)

h 代码得到的结果：<r, φ1, φ2, φ3>
- r 是模（abs(v)）
- φ 表示角坐标

In [41]:
format(Vector([-1, -1, -1, -1]), 'h')

'<2.0, 2.0943951023931957, 2.186276035465284, 3.9269908169872414>'

In [42]:
format(Vector([2, 2, 2, 2]), '.3eh')

'<4.000e+00, 1.047e+00, 9.553e-01, 7.854e-01>'

In [43]:
format(Vector([0, 1, 0, 0]), '0.5fh')

'<1.00000, 1.57080, 0.00000, 0.00000>'

动态类型是指在运行时检查类型，因为方法签名和变量没有静态类型信息。

在 Python 文档中，“文件类对象”这样的表述通常说的就是协议，意思是：“行为基本与文件一致，实现了部分文件接口，满足上下文相关需求的东西。”

不要为了满足过度设计的接口契约和让编译器开心，而去实现不需要的方法，我们要遵守 [KISS 原则](https://zh.wikipedia.org/zh/KISS%E5%8E%9F%E5%88%99)
- Keep It Simple, Stupid

如果使用 lambda ，或许就不应该使用列表推导，过滤除外

In [44]:
mylist = [[1, 2, 3], [40, 50, 60], [9, 8, 7]]

# 比使用 reduce 函数更易阅读
# 避免空序列问题，sum([]) 的结果是 0
sum(sub[1] for sub in mylist)

60